# COMP5046 Assignment 1
LiangChun Yu   
SID: 480317999   
Unicode: liyu7088




# Readme


Each step is commented through the process.

***Visualising the comparison of different results is a good way to justify your decision.***

# 1 - Data Preprocessing (Personality chat datasets)

## 1.1. Download Dataset (Personality chat datasets)

In [0]:
# Impporting packages, and the raw data for the assignment
import nltk
nltk.download('punkt')
import pandas as pd
import re
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import tensorflow as tf

!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


id = '1slWYfNpATxnHg_11_XMWrzHOdxW8iHcH'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_professional.tsv') 

id = '16KDzEPGGJYuS97EZ_kU82y37y0qWVwb9'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_friend.tsv') 

id = '1d08B982HlHHGZ_-ZnS04Dt30HZhFPsdc'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('qna_chitchat_the_comic.tsv') 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 1.2. Preprocess data (Personality chat datasets)

###A function is written to pre-process three of the data files. Through the function, we get the unique word list, sequence data, maximum input words(for further padding needs), and num_dic.###

###Note that only the question is preprocessed, decapitalisation and remove punctuations. Answers remain the same since we're doing a N to 1 model.###

### After running the function three times, we store all the data in separate dictionaries, which is easier to call in further process.###

In [0]:
# Please comment your code

# read and preprocess input data, return unique words list
def preprocess_data(file_name):
  file_name = file_name
  df = pd.read_csv(file_name, sep="\t")
  
  seq_data = []
  whole_words = []
  unique_words = []
  max_input_words = 0
  df = pd.read_csv(file_name, sep="\t")
  
# Read in text one row at a time, remove punctuation and decpitalize
# text for all questions.
  for index, row in df.iterrows():
    question = row['Question']
    answer = row['Answer']
    seq_data.append([question, answer])
    
    question = re.sub(r'[^\w\s]','', question.lower())
    tokenized_q = nltk.tokenize.word_tokenize(question)
    
    whole_words += tokenized_q
    whole_words.append(answer)
    max_input_words = max(len(tokenized_q), max_input_words)
  unique_words = list(set(whole_words))
  unique_words.append('_B_')
  unique_words.append('_E_')
  unique_words.append('_P_')
  unique_words.append('_U_')
  unique_words.sort()
# Sort words in a order so everytime we generate, we have the same order
  num_dic = {n:i for i,n in enumerate(unique_words)}
  return unique_words, seq_data, max_input_words, num_dic

d_comic, seq_comic, max_word_comic, num_dic_comic = preprocess_data('qna_chitchat_the_comic.tsv')
d_friend, seq_friend, max_word_friend, num_dic_friend = preprocess_data('qna_chitchat_the_friend.tsv')
d_professional, seq_professional, max_word_professional, num_dic_professional = preprocess_data('qna_chitchat_the_professional.tsv')


### Store all the genearated data in dictionaries, making it easier to call later
num_dic = {'comic':num_dic_comic, 'friend':num_dic_friend, 'professional':num_dic_professional}
# Store all num dics in a nested dictionary.
max_input_words = max(max_word_comic, max_word_friend, max_word_professional)
# To simplify the code, we just take the max words of the three models(though they're all the same)
dic_len_models = {'comic':len(num_dic['comic']), 'friend':len(num_dic['friend']), 'professional':len(num_dic['professional'])}
# And we have a dic of the dic_len of three modesl
unique_word_dic = {'comic':d_comic,'friend':d_friend, 'professional':d_professional}
# Store different unique_word list in a dict


# 2 - Model Implementation

## 2.1. Word Embeddings

*You are required to describe which model was implemented (i.e. Word2Vec with CBOW, FastText with SkipGram, etc.) with justification of your decision *

###We use Word2Vec SkipGram for our word embeddings. From lectures and reserach, skipgram normally performs better than CBOW, and Word2Vec embeddings model can be generated through a one-liner function, and model trained in advanve, which is convenient for the assignment.###

### 2.1.1. Download Dataset for Word Embeddings



The same dataset is used, making it easier to process and build models.

### 2.1.2. Data Preprocessing for Word Embeddings

 We decapitalize, remove puctuations, remove numbers, then tokenize word, to generate a tokens list for the word embedding.

In [0]:
# Please comment your code

# Generate token list for word embeddings.
def process_data_embedding(seq_data):
  seq_data = seq_data
  
  normalized_text = []
  tok_list = []
  for seq in seq_data:
    question = seq[0]
    answer = seq[1]
# preprocess data, and generate tokens. Do not tokenize answers.    
    question = re.sub(r"[^a-z0-9]+", " ", question.lower())
    question = re.sub(r'[^\w\s]','',question)
    tokens = word_tokenize(question)
    tokens.append(answer)
    tok_list.append(tokens)
  
  tok_list.append(['_B_','_E_','_P_','_U_']) 
      
  return tok_list

### 2.1.3. Build Word Embeddings Model

*You are required to describe how hyperparameters were decided with justification of your decision.*

###We keep most of the parameters same(SkipGram), only changing min_count to 1, since we want to keep all words in the word2vec vocabulary. Vector size of 100 causes the training to be slower, but it does causes larger costs in the further seq2seq training process, so we keep the size as 100.###

In [0]:
# Word2Vec models are stored in the models dictionary

models = {}

models['comic'] = Word2Vec(sentences=process_data_embedding(seq_comic), size=100, window=5, min_count=1, workers=4, sg=1)
models['friend'] = Word2Vec(sentences=process_data_embedding(seq_friend), size=100, window=5, min_count=1, workers=4, sg=1)
models['professional'] = Word2Vec(sentences=process_data_embedding(seq_professional), size=100, window=2, min_count=1, workers=4, sg=1)


In [0]:
a = models['comic']
print(len(a.wv.vocab))
print(a.wv.vocab['hi'])
print(type(a.wv.vocab['hi']))
print(a.wv.vocab['hi'].shape)



617
Vocab(count:6, index:125, sample_int:4294967296)
<class 'gensim.models.keyedvectors.Vocab'>


AttributeError: ignored

### 2.1.4. Train Word Embeddings Model

In [0]:
# Please comment your code

# Word2vec models already trained.

(45701, 68240)

### 2.1.5. Save Word Embeddings Model

In [0]:
# Please comment your code
# Upload model to google drive, and storing the model ids in model_id
from gensim.test.utils import common_texts, get_tmpfile

model_id = {}
folder_id = "1YNUp5NkhlojOYio279f6DnVA71jBDUkP"
# store in a specific folder of google drive
for key in models:
  path = get_tmpfile("word2vec.model")
  models[key].save("word2vec.model")
  model_name = key 

  uploaded = drive.CreateFile({'title': model_name, 'parents':[{
      'kind':'drive#fileLink','id':folder_id}]})
  uploaded.SetContentFile('word2vec.model')
  uploaded.Upload()
  model_id[key] = uploaded.get('id')
  print('Uploaded file with ID {}'.format(uploaded.get('id')))
print(model_id)

Uploaded file with ID 1bUZaUs8Z4xc31xHl7R8vAzgSqNUOASfT
Uploaded file with ID 1uV5vF9THBZKv6yT3rui6BYnkj5KvkBx9
Uploaded file with ID 1HRzTl5E4D-LNYqZ1OLUw6MONKAwevhsB
{'comic': '1bUZaUs8Z4xc31xHl7R8vAzgSqNUOASfT', 'friend': '1uV5vF9THBZKv6yT3rui6BYnkj5KvkBx9', 'professional': '1HRzTl5E4D-LNYqZ1OLUw6MONKAwevhsB'}


### 2.1.6. Load Word Embeddings Model

In [0]:
# Call stored file ids, and download the models, store then in a new dict
download_models = {'comic':'','friend':'','professional':''}
for key in download_models:  
  id = model_id[key]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(key) 
  download_models[key] = Word2Vec.load(key)
print(download_models)

{'comic': <gensim.models.word2vec.Word2Vec object at 0x7f59a2b0db70>, 'friend': <gensim.models.word2vec.Word2Vec object at 0x7f59a17e57f0>, 'professional': <gensim.models.word2vec.Word2Vec object at 0x7f59a13244a8>}


## 2.2. Seq2Seq model

### 2.2.1. Apply/Import Word Embedding Model

We call the trainied word embeddings model downloaded above.

### 2.2.2. Build Seq2Seq Model


###We have global variables generated in the very first preprocess step.In this step we get the word2vec vectors for the input and outputbatch, target batch on the other hand is a num_dic series of numbers.Inside the following functions, we add paddings for questions, making every input batch the same window size. And we mark the beggining and end of the data of each sequence('_B_', '_E_'). '_U_' is added for words not in the word2vec vocabulary.###

###Hyperparameters are remained mostly same from tutorials, we change number of epochs to 2500 since it is quite time consuming. n-class here is 100, which is the size of the word2vec vector. The session is also saved and uploaded to google drive in this step.###

### We train three models separately in this step, since errors keep occuring while buiding models within a function.###

In [0]:
# Please comment your code

# We have seq_comic, seq_friend, seq_professional three sequence datas
# max_input_words is a global variable
# dic_len_models stores three dic_len of three modesl
def get_vectors_q(sentence, model):
  sentence = sentence
  model_local = model # the model where we retrieve the vectors
  
   # add paddings 
  tokenized_sentence = sentence.split()
  diff = max_input_words - len(tokenized_sentence)  
  for x in range(diff):
    tokenized_sentence.append('_P_')
  
  # get vectors of each token
  ids = []
  for tok in tokenized_sentence:
# check if token in word embedding vocabulary
# then add the vector into ids    
    if tok in model_local.wv.vocab:
      ids.append(model_local[tok])
    else:
      ids.append(model_local['_U_'])
    
  return ids


def get_vectors_a(sentence, model):
  model_local = model
  ids = []
  if sentence in model_local.wv.vocab:
    ids.append(model_local[sentence])
  else:
    ids.append(model_local['_U_'])
  
  return ids


In [0]:
def make_batch(seq_data, model_name):
  model = download_models[model_name]
  num_dic_local = num_dic[model_name]
  
  input_batch = []
  output_batch = []
  target_batch = []
  for seq in seq_data:
    input_batch.append(get_vectors_q(seq[0], model))
    
    output_data = []
    output_data.append(model['_B_'])
    output_data += get_vectors_a(seq[1], model)    
    output_batch += [output_data]
    
    target = []
    if seq[1] in num_dic_local:
      target.append(num_dic_local[seq[1]])
    else:
      target.append(num_dic_local['_U_'])
    target.append(num_dic_local['_E_'])
    target_batch += [target]
  
  return input_batch, output_batch, target_batch


### 2.2.3. Train Seq2Seq Model

In [0]:
# Build, train, and save model. Separately for three personalities.

# Build model for 'comic'
dic_len = dic_len_models['comic']

learning_rate = 0.002
n_hidden = 128

n_class = dic_len
n_input = 100 # the shape of the word2vec vector

### Neural Network Model
tf.reset_default_graph()

# encoder/decoder shape = [batch size, time steps, input size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])

# target shape = [batch size, time steps]
targets = tf.placeholder(tf.int64, [None, None])


# Encoder Cell
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
# Decoder Cell
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # [IMPORTANT] Setting enc_states as inital_state of decoder cell
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

seq2_model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=seq2_model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


# Save session
saver = tf.train.Saver() 
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# Generate a batch data
input_batch, output_batch, target_batch = make_batch(seq_comic, 'comic')

total_epoch = 2500

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')
saver.save(sess, 'comic_final')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Epoch: 0001 cost = 6.439291
Epoch: 0101 cost = 2.269004
Epoch: 0201 cost = 2.253289
Epoch: 0301 cost = 2.243768
Epoch: 0401 cost = 2.246928
Epoch: 0501 cost = 2.238682
Epoch: 0601 cost = 2.241033
Epoch: 0701 cost = 2.232227
Epoch: 0801 cost = 2.239636
Epoch: 0901 cost = 2.241529
Epoch: 1001 cost = 2.239633
Epoch: 1101 cost = 2.240237
Epoch: 1201 cost = 2.237710
Epoch: 1301 cost = 2.244757
Epoch: 1401 cost = 2.242025
Epoch: 1501 cost = 2.236592
Epoch: 1601 cost = 2.242176
Epoch: 1701 cost = 2.243438
Epoch: 1801 cost = 2.235209
Epoch: 1901 cost = 2.238647
Epoch: 2001 cost = 2.238959
Epoch: 2101 cost = 2.237427
Epoch: 2201 cost = 1.993019
Epoch: 2301 cost = 1.738505
Epoch: 2401 cost = 1.552091
Epoch: 2500 cost = 1.467875
Training completed


'comic_final'

In [0]:
# upload comic seq model

folder_id = "1YNUp5NkhlojOYio279f6DnVA71jBDUkP"


uploaded = drive.CreateFile({'title': 'checkpoint', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./checkpoint')
uploaded.Upload()
comic_checkpoint_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'comic_meta', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./comic_final.meta')
uploaded.Upload()
comic_meta_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'comic_index', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./comic_final.index')
uploaded.Upload()
comic_index_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'comic_data', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./comic_final.data-00000-of-00001')
uploaded.Upload()
comic_data_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))



Uploaded file with ID 1ckhsS8AOc12jSQJCn6WSWrrM1-KVXbuf
Uploaded file with ID 1UZE7mm3Ij4RlFZSczFpFikkpa8scLdrB
Uploaded file with ID 1GVxACQ-lD8WLB9J06qYhJbmG_0_vteKm
Uploaded file with ID 1S_co064QT3-iS-FV8J8_zsnrpR6DedQ-


In [0]:
# Build "friend" seq model

dic_len = dic_len_models['friend']

learning_rate = 0.002
n_hidden = 128
n_class = dic_len
n_input = 100 # the shape of the word2vec vector

tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# Encoder Cell
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)
    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
# Decoder Cell
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)

seq2_model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=seq2_model, labels=targets))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)


saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_friend, 'friend')
total_epoch = 2500

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')

saver.save(sess, 'friend_final')


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Epoch: 0001 cost = 6.439257
Epoch: 0101 cost = 2.266796
Epoch: 0201 cost = 2.249656
Epoch: 0301 cost = 2.247743
Epoch: 0401 cost = 2.246700
Epoch: 0501 cost = 2.247783
Epoch: 0601 cost = 2.243129
Epoch: 0701 cost = 2.245524
Epoch: 0801 cost = 2.243223
Epoch: 0901 cost = 2.247496
Epoch: 1001 cost = 2.246047
Epoch: 1101 cost = 2.249654
Epoch: 1201 cost = 2.248470
Epoch: 1301 cost = 2.244512
Epoch: 1401 cost = 2.248358
Epoch: 1501 cost = 2.247425
Epoch: 1601 cost = 2.249988
Epoch: 1701 cost = 2.242080
Epoch: 1801 cost = 1.978019
Epoch: 1901 cost = 1.801824
Epoch: 2001 cost = 1.661260
Epoch: 2101 cost = 1.546105
Epoch: 2201 cost = 1.462118
Epoch: 2301 cost = 1.366052
Epoch: 2401 cost = 1.340990
Epoch: 2500 cost = 1.269823
Training completed


'friend_final'

### 2.2.4. Save Seq2Seq Model

In [0]:
# upload seq model for friend
folder_id = "1YNUp5NkhlojOYio279f6DnVA71jBDUkP"
 
uploaded = drive.CreateFile({'title': 'friend_checkpoint', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./checkpoint')
uploaded.Upload()
friend_checkpoint_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'friend_meta', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./friend_final.meta')
uploaded.Upload()
friend_meta_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'friend_index', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./friend_final.index')
uploaded.Upload()
friend_index_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'friend_data', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./friend_final.data-00000-of-00001')
uploaded.Upload()
friend_data_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))




Uploaded file with ID 12MFZ7JWiENkuGBID6FmDjt0JiIfGC4eP
Uploaded file with ID 1mEI8h0XtGHugVIovh1PXGRn7CSi6vNqd
Uploaded file with ID 16cKUgd-XXN-He5Xqeq2yPJFtnDDPLI5e
Uploaded file with ID 1FvcXL6NB3giwxXlVwAUmexV1D3oFxtJ-


In [0]:
# Train Professional Seq Model

dic_len = dic_len_models['professional']

learning_rate = 0.002
n_hidden = 128
n_class = dic_len
n_input = 100 # the shape of the word2vec vector


tf.reset_default_graph()
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])
targets = tf.placeholder(tf.int64, [None, None])

# Encoder Cell
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                            dtype=tf.float32)
# Decoder Cell
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                            initial_state=enc_states,
                                            dtype=tf.float32)
    
seq2_model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=seq2_model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_professional, 'professional')

total_epoch = 2500
for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost],
                       feed_dict={enc_input: input_batch,
                                  dec_input: output_batch,
                                  targets: target_batch})
    if epoch % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1),
              'cost =', '{:.6f}'.format(loss))

print('Epoch:', '%04d' % (epoch + 1),
      'cost =', '{:.6f}'.format(loss))
print('Training completed')

saver.save(sess, 'professional_final')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Epoch: 0001 cost = 6.442017
Epoch: 0101 cost = 2.273122
Epoch: 0201 cost = 2.253241
Epoch: 0301 cost = 2.243272
Epoch: 0401 cost = 2.243923
Epoch: 0501 cost = 2.238600
Epoch: 0601 cost = 2.242177
Epoch: 0701 cost = 2.243120
Epoch: 0801 cost = 2.243827
Epoch: 0901 cost = 2.237396
Epoch: 1001 cost = 2.244642
Epoch: 1101 cost = 2.242625
Epoch: 1201 cost = 2.240697
Epoch: 1301 cost = 2.241271
Epoch: 1401 cost = 2.242280
Epoch: 1501 cost = 2.241081
Epoch: 1601 cost = 2.240868
Epoch: 1701 cost = 2.237412
Epoch: 1801 cost = 2.242924
Epoch: 1901 cost = 2.245448
Epoch: 2001 cost = 2.236545
Epoch: 2101 cost = 2.239727
Epoch: 2201 cost = 2.237613
Epoch: 2301 cost = 2.236903
Epoch: 2401 cost = 2.239925
Epoch: 2500 cost = 2.226492
Training completed


'professional_final'

In [6]:
# Save Model of Professional
folder_id = "1YNUp5NkhlojOYio279f6DnVA71jBDUkP"
 
uploaded = drive.CreateFile({'title': 'professional_checkpoint', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./checkpoint')
uploaded.Upload()
professional_checkpoint_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'professional_meta', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./professional_final.meta')
uploaded.Upload()
professional_meta_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'professional_index', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./professional_final.index')
uploaded.Upload()
professional_index_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))

uploaded = drive.CreateFile({'title': 'professional_data', 'parents':[{
    'kind':'drive#fileLink','id':folder_id}]})
uploaded.SetContentFile('./professional_final.data-00000-of-00001')
uploaded.Upload()
professional_data_id = uploaded.get('id')
print('Uploaded file with ID {}'.format(uploaded.get('id')))



NameError: ignored

### 2.2.5. Load Seq2Seq Model

In [0]:
# Function to download seq2seq model

def download_seq_model(model_name):
  if model_name == 'friend':
    id = friend_checkpoint_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('friend_checkpoint')

    id = friend_meta_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('friend_meta')

    id = friend_index_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('friend_index')

    id = friend_data_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('friend_data')
  elif model_name == 'comic':
    id = comic_checkpoint_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('checkpoint')

    id = comic_meta_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('comic_meta')

    id = comic_index_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('comic_index')

    id = comic_data_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('comic_data')
  elif model_name == 'professional':
    id = professional_checkpoint_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('professional_checkpoint')

    id = professional_meta_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('professional_meta')

    id = professional_index_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('professional_index')

    id = professional_data_id
    downloaded = drive.CreateFile({'id':id})
    downloaded.GetContentFile('professional_data')
  else:
    print('Please enter correct personality.')
       

In [0]:
# Get respond for the input text, we build the model again inside, so the variables can function

def answer(sentence, model_name):
  unique_word = unique_word_dic[model_name]
  dic_len = dic_len_models[model_name]
  tf.reset_default_graph()
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    learning_rate = 0.002
    n_hidden = 128
    n_class = dic_len
    n_input = 100

    enc_input = tf.placeholder(tf.float32, [None, None, n_input])
    dec_input = tf.placeholder(tf.float32, [None, None, n_input])
    targets = tf.placeholder(tf.int64, [None, None])

    # Encoder Cell
    with tf.variable_scope('encode'):
        enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

        outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input,
                                                dtype=tf.float32)
    # Decoder Cell
    with tf.variable_scope('decode'):
        dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
        dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

        # [IMPORTANT] Setting enc_states as inital_state of decoder cell
        outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input,
                                                initial_state=enc_states,
                                                dtype=tf.float32)
    model = tf.layers.dense(outputs, n_class, activation=None)
    cost = tf.reduce_mean(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    logits=model, labels=targets))
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    restore_name = './'+model_name+'_final'
    saver = tf.train.Saver()
    saver.restore(sess, restore_name)
    
    seq_data = [sentence, '_U_']
    input_batch, output_batch, target_batch = make_batch([seq_data],model_name)
    
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction,
                      feed_dict={enc_input: input_batch,
                                 dec_input: output_batch,
                                 targets: target_batch})

    # convert index number to actual token 
    #unique_word_list = unique_word
    decoded = [unique_word[i] for i in result[0]]
        
    # Remove anything after '_E_'        
    if "_E_" in decoded:
        end = decoded.index('_E_')
        translated = ' '.join(decoded[:end])
    else :
        translated = ' '.join(decoded[:])
    
    return translated



# 3 - Evaluation (Running chatbot)

###In this step, we write start_chat, change_personality, and save_chat_log functions for the chatbot. Only by typing 'exit' can the user finish the chat. 'change' is used to change personality. The chat is also recorded in two separate lists, and will be written to a text file after the chat is terminated.

## 3.1. Start chatting

In [0]:
# Start Chat

def start_chat():
  print("Chatbot: Welcome. Please choose a personality.")  
  personality = change_personality()

  temp = ''
  change_option = ['change', 'exit']
  type_list = []
  respond_list = []
  while temp not in change_option:
    word = input("You: ")
    temp = word.lower()
    respond = answer(word, personality)
    print("Chatbot: "+ respond)
    type_list.append(word)
    respond_list.append(respond)
    
  return type_list, respond_list, temp



## 3.2. Change Personality

*Explain how to change personality (What is the command for changing personality?). *

In [0]:
# Please comment your code
def change_personality():
  print("")
  print("Chatbot: Please choose a personality.")  
  word = input("You: ").lower()
  #word = word.lower()
  person_list = ['comic', 'friend', 'professional']
  while word not in person_list:    
    print("Please type a correct personality.")
    word = input("You: ").lower()
  print("Chatbot: Okay, let's chat!")
  return word



## 3.3. Save chat log

In [0]:
# Chat record written and saved to local directory.
from google.colab import files

def save_chat_log(input_file, output_file):
  f = open("chat_log.txt", "w")
  for i in range(len(input_file)):
    f.write("You: "+input_file[i]+"\n")
    f.write("Chatbot: "+output_file[i]+"\n")
  f.close()
  files.download('chat_log.txt')
  


## 3.4. End chatting

In [0]:
# Please comment your code

# When user types "exit", the chat is ended.

## 3.5. Execute program

***Please make sure your program  is running properly.***

***Functions for downloading (from Google Drive) and loading models (both word embeddings and Seq2Seq) need to be called!*** 


### 3.5.1. Execute program - training mode

*Please include lines to train the bot.*

In [5]:
# Download the seq_models

download_seq_model('comic')
download_seq_model('friend')
download_seq_model('professional')



NameError: ignored

### 3.5.2. Execute program - chatting mode

*Please include lines to start chatting with the bot.*

In [1]:
# To start chat, we call the start_chat function.


list_of_input = []
list_of_output = []

type_list, respond_list, change = start_chat()

# only 'change' or 'exit' will jump out of the loop

while change == 'change':
  list_of_input += type_list
  list_of_output += respond_list
  type_list, respond_list, change = start_chat()
# the only way to exit loop is when change = 'exit'

print("Chatbot: Thank you! See you again.")

save_chat_log(list_of_input, list_of_output)



NameError: ignored

## Object Oriented Programming codes here

*If you have multiple classes use multiple code snippets to add them.*

In [0]:
# If you used OOP style, use this sectioon